In [2]:
import importlib
import utilities
importlib.reload(utilities)
from utilities import get_files_from_dir, chunk_list
from facenet_pytorch import InceptionResnetV1
from PIL import Image
import torch
from torchvision import transforms
import tqdm
importlib.reload(tqdm)

/Users/shirabroner/DataScience/venv/Default/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


<module 'tqdm' from '/Users/shirabroner/DataScience/venv/Default/lib/python3.9/site-packages/tqdm/__init__.py'>

In [3]:
faces_dir = 'faces/*'
faces_files = get_files_from_dir(faces_dir)

In [4]:
device = 'mps'

In [5]:
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to a tensor
    #transforms.Resize((256, 256))
    # Add other transformations if necessary
    # e.g., transforms.Resize((256, 256)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
with torch.no_grad():
    x = []
    y = []
    for i, face_file in tqdm.tqdm(enumerate(faces_files), desc='Processing', total=len(faces_files)):
        y.append(face_file)
        x.append(resnet(transform(Image.open(face_file)).to(device).unsqueeze(0)).cpu())

torch.save({'x': torch.concat(x), 'y':y}, 'face_embedding.pt')

Processing: 100%|██████████| 11806/11806 [06:56<00:00, 28.31it/s]


In [9]:
dataset = torch.load('face_embedding.pt')

In [10]:
dataset['x'].numpy()

tensor([[-0.0126,  0.0387, -0.0037,  ..., -0.0064,  0.0009,  0.0218],
        [-0.0710,  0.0005, -0.0321,  ...,  0.0048,  0.0079,  0.0107],
        [-0.0219,  0.1007, -0.0235,  ...,  0.0121,  0.1069,  0.0359],
        ...,
        [-0.0061,  0.0281,  0.0526,  ..., -0.0446, -0.0025,  0.0056],
        [ 0.0155,  0.0158,  0.0326,  ...,  0.0013, -0.0165, -0.0247],
        [-0.0654,  0.0481, -0.0925,  ..., -0.0097,  0.1020, -0.0009]])